# Also Watched
* Attaches a column to the recommendations about what other viewers have watched

In [ ]:
@memoize read_params_memoized(x) = read_params(x);

In [ ]:
function get_also_watched(watch_similarity, related_series, item, allowed_items)
    W = watch_similarity
    R = related_series
    related_items = Set(R[:, item+1].nzind .- 1)
    also_watched = nothing
    watch_rate = -Inf
    for x in allowed_items
        if x in related_items
            continue
        end
        rate = W[x + 1, item + 1]
        if rate > watch_rate
            watch_rate = rate
            also_watched = x
        end
    end
    also_watched
end;

In [ ]:
function attach_also_watched!(rec_df, allowed_df, medium)
    W = read_params_memoized("$medium/all/WatchSimilarity")["S"]
    R = read_params_memoized("$medium/all/RelatedSeries/similarity_matrix")["S"]
    also_watched = [get_also_watched(W, R, x, allowed_df.uid) for x in rec_df.uid]
    links = [get_link(medium, source, x) for x in also_watched]
    if medium == "anime"
        col = :viewers_also_watched
    elseif medium == "manga"
        col = :readers_also_read
    else
        @assert false
    end
    rec_df[:, col] = links
    rec_df
end;